In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.utils.data import Dataset, DataLoader

from sklearn import tree, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
# np.set_printoptions(precision=8, suppress=True)

In [3]:
# %matplotlib inline
%matplotlib qt

In [4]:
writer = SummaryWriter('runs/log')

In [5]:
n_epochs = 40 #100   # number of epochs to run
batch_size = 10  # size of each batch

In [6]:
# X

In [7]:
# X_train[:]

In [15]:
class FutureTrendModule(nn.Module):
    def __init__(self):
        super(FutureTrendModule, self).__init__()
        
        # input batch_size 300
        self.fc1 = nn.Sequential(
            nn.Linear(900, 128),
        )
        
        # input batch_size 1 128
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(True),
#             nn.AvgPool1d(3, 2),
            nn.MaxPool1d(3, 2),
        )
        
        # input batch_size 64 64
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=2),
            nn.ReLU(True),
#             nn.AvgPool1d(3, 2),
            nn.MaxPool1d(3, 2),
        )
        
        # input batch_size 32 32
        self.conv3 = nn.Sequential(
            nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=2),
            nn.ReLU(True),
#             nn.AvgPool1d(3, 2),
            nn.MaxPool1d(3, 2),
        )
        
        self.dropout = nn.Dropout(0.2)
        
        # input batch_size 256
        self.output_fc1 = nn.Linear(256, 128)
        # input batch_size 128
        self.output_fc2 = nn.Linear(128, 1)

    def forward(self, x):
#         return self.model(x)
        fc1 = self.fc1(x).view(-1, 1, 128)
        conv1 = self.conv1(fc1)
        conv2 = self.conv2(conv1)
        conv3 = self.conv3(conv2)
        dropout = self.dropout(conv3.view(-1, 256))
        out_fc1 = self.output_fc1(dropout)
        outs = self.output_fc2(out_fc1)
        return outs, fc1, conv1, conv2, conv3, dropout, out_fc1
    

In [29]:
model = FutureTrendModule().to(0)

In [30]:
# loss_fn = nn.MSELoss().to(0)  # mean square error
loss_fn = nn.SmoothL1Loss().to(0)
# loss_fn = nn.CrossEntropyLoss().to(0)
# optimizer = optim.Adam(model.parameters(), lr=0.000001)
optimizer = optim.SGD(model.parameters(), lr = 0.000001, momentum=0.09)

In [31]:
# SAVE_PATH = 'D:/option/datas'
# SAVE_PATH = 'D:/option/l8'
SAVE_PATH = 'D:/option/l8_5mean_classification'

In [ ]:
# Hold the best model
best_mse = np.inf   # init to infinity
best_weights = None
history = []


step = 0
# training loop
for epoch in range(n_epochs):
    model.train()
    
#     total_size = 0
    
#     for ix in range(0, len(dfs_x)-20):
    for f in os.listdir(SAVE_PATH):
        if not f.startswith('SHFE.rb'):
            continue
        
        store = pd.HDFStore(os.path.join(SAVE_PATH, f))
        X = normalize(store['X'].values)
#         y = store['y'].values * 100
        y = store['y'].values[:,1]
        store.close()
        
#         X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=False)
        X_train = torch.tensor(X, dtype=torch.float32, device=0)
        y_train = torch.tensor(y, dtype=torch.float32, device=0).reshape(-1, 1)
        
        total_size = X_train.shape[0]
        batch_start = torch.arange(0, total_size, batch_size)
        
        running_loss = 0.0
        for i, start in enumerate(batch_start, 1):
            # take a batch
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
#             y_pred, fc1, conv1, conv2, conv3, out_fc1 = model(X_batch) # .reshape(-1, 1, 300)
            y_pred, fc1, conv1, conv2, conv3, dropout, out_fc1 = model(X_batch)
            
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress

            running_loss += loss.item() * X_batch.shape[0]

#             step = epoch * len(batch_start) + i
#             writer.add_scalar('loss', loss.item(), step)

#             if i % 5000 == 0:

        step += total_size
        
        writer.add_scalar('loss/loss', loss.item(), step)
        writer.add_scalar('loss/total_loss', running_loss / (total_size), step)
        
        writer.add_histogram('deep/01_x', torchvision.utils.make_grid(X_batch).data.cpu(), step)
        writer.add_histogram('deep/02_fc1', torchvision.utils.make_grid(fc1).data.cpu(), step)
        writer.add_histogram('deep/03_conv1', torchvision.utils.make_grid(conv1).data.cpu(), step)
        writer.add_histogram('deep/04_conv2', torchvision.utils.make_grid(conv2).data.cpu(), step)
        writer.add_histogram('deep/05_conv3', torchvision.utils.make_grid(conv3).data.cpu(), step)
        writer.add_histogram('deep/06_dropout', torchvision.utils.make_grid(dropout).data.cpu(), step)
        writer.add_histogram('deep/07_out_fc1', torchvision.utils.make_grid(out_fc1).data.cpu(), step)

        writer.add_histogram('deep/09_pred', torchvision.utils.make_grid(y_pred).data.cpu(), step)

#         print('Epoch: [{}/{}], [{}/{}], Loss: {:.8f}'.format(epoch + 1, n_epochs, start+10, total_size, running_loss / (X_batch.shape[0] * i)))
#         print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, ix, running_loss / (total_size)))
        print('Epoch: [{}/{}], {}, Loss: {:.8f}'.format(epoch + 1, n_epochs, f, running_loss / (total_size)))
    
#     for ix in range(len(dfs_x)-20, len(dfs_x)):
#         X_test = torch.tensor(dfs_x[ix].values, dtype=torch.float32, device=0)
#         y_test = torch.tensor(dfs_y[ix].values, dtype=torch.float32, device=0).reshape(-1, 1)
        
#         batch_test_start = torch.arange(0, len(X_test), batch_size)
        
#         # evaluate accuracy at end of each epoch
#         model.eval()
#         eval_loss = 0.0
#         for i, start in enumerate(batch_test_start, 1):
#             X_batch_test = X_test[start:start+batch_size]
#             y_batch_test = y_test[start:start+batch_size]

#             y_pred, fc1, conv1, conv2, conv3, out_fc1 = model(X_batch_test)
#             loss = loss_fn(y_pred, y_batch_test).cpu().detach().numpy()

#             eval_loss += loss.item() * X_batch_test.shape[0]

#         eval_loss = eval_loss / X_test.shape[0]
#         print('eval loss {:.8f}'.format(eval_loss))
#         history.append(eval_loss)
#         if eval_loss < best_mse:
#             best_mse = eval_loss
#             best_weights = copy.deepcopy(model.state_dict())
 
# # restore model and return best accuracy
# model.load_state_dict(best_weights)

Epoch: [1/40], SHFE.rb.2022-12-02.41243.h5, Loss: 2.15137022
Epoch: [1/40], SHFE.rb.2022-12-05.41281.h5, Loss: 2.27118863
Epoch: [1/40], SHFE.rb.2022-12-06.40848.h5, Loss: 2.01426230
Epoch: [1/40], SHFE.rb.2022-12-07.41234.h5, Loss: 2.27519231
Epoch: [1/40], SHFE.rb.2022-12-08.41207.h5, Loss: 2.10357657
Epoch: [1/40], SHFE.rb.2022-12-09.41358.h5, Loss: 2.42550006
Epoch: [1/40], SHFE.rb.2022-12-12.41249.h5, Loss: 2.23307048
Epoch: [1/40], SHFE.rb.2022-12-13.41244.h5, Loss: 2.38360830
Epoch: [1/40], SHFE.rb.2022-12-14.41187.h5, Loss: 2.19051598
Epoch: [1/40], SHFE.rb.2022-12-15.41351.h5, Loss: 2.63979245
Epoch: [1/40], SHFE.rb.2022-12-16.41382.h5, Loss: 2.72954196
Epoch: [1/40], SHFE.rb.2022-12-19.41305.h5, Loss: 2.59261314
Epoch: [1/40], SHFE.rb.2022-12-20.41323.h5, Loss: 2.47904116
Epoch: [1/40], SHFE.rb.2022-12-21.41294.h5, Loss: 2.46065731
Epoch: [1/40], SHFE.rb.2022-12-22.41297.h5, Loss: 2.54562134
Epoch: [1/40], SHFE.rb.2022-12-23.41363.h5, Loss: 2.49387843
Epoch: [1/40], SHFE.rb.2

Epoch: [2/40], SHFE.rb.2022-12-20.41323.h5, Loss: 2.48019772
Epoch: [2/40], SHFE.rb.2022-12-21.41294.h5, Loss: 2.46013906
Epoch: [2/40], SHFE.rb.2022-12-22.41297.h5, Loss: 2.54279517
Epoch: [2/40], SHFE.rb.2022-12-23.41363.h5, Loss: 2.49524128
Epoch: [2/40], SHFE.rb.2022-12-26.41245.h5, Loss: 2.26778931
Epoch: [2/40], SHFE.rb.2022-12-27.41203.h5, Loss: 2.29010029
Epoch: [2/40], SHFE.rb.2022-12-28.41268.h5, Loss: 2.16378984
Epoch: [2/40], SHFE.rb.2022-12-29.41263.h5, Loss: 2.12876244
Epoch: [2/40], SHFE.rb.2022-12-30.26898.h5, Loss: 2.24706234
Epoch: [2/40], SHFE.rb.2023-01-03.41159.h5, Loss: 1.88083559
Epoch: [2/40], SHFE.rb.2023-01-04.41291.h5, Loss: 2.34520742
Epoch: [2/40], SHFE.rb.2023-01-05.41261.h5, Loss: 2.23900049
Epoch: [2/40], SHFE.rb.2023-01-06.41378.h5, Loss: 2.59752767
Epoch: [2/40], SHFE.rb.2023-01-09.41028.h5, Loss: 1.98125561
Epoch: [2/40], SHFE.rb.2023-01-10.41074.h5, Loss: 2.01208957
Epoch: [2/40], SHFE.rb.2023-01-11.41239.h5, Loss: 1.87653834
Epoch: [2/40], SHFE.rb.2

In [2]:
y_pred

NameError: name 'y_pred' is not defined

In [28]:
y_test

NameError: name 'y_test' is not defined

In [30]:
torch.save(model.state_dict(), 'D:/option/models/model_0608_01.pt')

In [29]:
print("MSE: %.2f" % best_mse)
print("RMSE: %.2f" % np.sqrt(best_mse))
plt.plot(history)
plt.show()

MSE: 0.14
RMSE: 0.38


In [16]:
np.array(history)

array([0., 0., 0.], dtype=float32)